# Phase II Model With Real Data
First attempt at a Phase II model, one which creases the master schedule and assigns students to courses simultaneously.

$S$ -- Set of all students

$C$ -- Set of all courses

$T$ -- Set of all periods {1,2,3,4,7,8} 

$I$ -- Set of all instructors

$R$ -- set of all rooms 

**Variables:**

$x_{i,j}$ for  $i \in S, j \in C$ -- Binary, 1 if student $i$ assigned to course $j$ 

$c_{j,t}$ for $j \in C, t \in T$ -- Binary, 1 if course $j$ to be offered in period $t$

$r_{j,s,t}$ for $j \in C$, $s \in R$ and $t \in T$ -- binary, describes if course $j$ to be held in room $s$ durring period $t$.

$u_{i,j,t}$ for $i \in S$, $j \in C$, $t \in T$ -- binary, used for constraint, no practical meaning.

** Parameters:**

$P_{i,j}$ -- Preference for student $i$ on course $j$

$S_i$ -- Seniority constant, e.g., higher for seniors

$D_{i,j}$ -- Binary, 1 if course $i$ and $j$ are in the same department, i.e., if they meet the same requirement, e.g., highschool math

$Ta_{i,j}$ -- Binary, 1 if teacher $i$ is teaching course $j$

$Cap_j$ -- Capacity of course $j$

$Min_j$ -- Minimum number of students needed for course $j$

$Db_j$ -- Indicates if course $j$ is a double period (1 or 0)


** Constraints: **

$\sum_{j} x_{i,j} =6 \quad \forall i \in S$ -- Says students can be assigned to six courses (full course load).

$ x_{i,j} \leq P_{i,j} \; \forall i \in S, j \in C$ -- Says students not assigned to courses they didn't preference. 


$x_{i,j} \geq \sum_{t} u_{i,j,t} \; \forall i \in S, j \in C$ -- Used to ensure $u$ takes correct value of $x$ and $c$

$c_{j,t} \geq u_{i,j,t} \; \forall i \in S, j \in C, t \in T$ -- This, in conjunction with the above are sufficient to say that no student is enrolled in more than one course per period. 

$ \text{lower bound}_{i,j} \leq \sum_{i \in C} D_{i,j}x_{k,i} \leq \text{upper bound}_{i,j} \; \forall k \in S, j \in C$ -- Says tha students take within a certain number of courses in the department, where the department is defined by the course $j$. ** new, but still likely unworkable department constraint--not in code **

$\sum_{t \in T} c_{j,t} = 1 \quad \forall j \in C$ -- Says that each course taught only once

$\sum_{i \in S} x_{i,j} \leq Cap_j \quad \forall j \in C$ -- Course capacity constraint

$\sum_{j \in C} c_{j,t} Ta_{k,j} \leq 1  \quad \forall k \in I, \forall t \in T$ -- Teacher constraint (a teacher can teach at most one course per period), where $Ta_{k,j}$ is a parameter, not a variable

*Double Period Constraints*---In terms of courses, we only add constraints if the parameter value $Db_j=1$, note, this *is* linear, as the parameter is not a variable: 
$c_{j,t} \leq c_{j+1, t+1} \; \forall j:Db_j =1$ -- cannot schedule the first half without also doing the second  
$c_{j,8} = 0, c_{j,4} = 0 \; \forall j:Db_j = 1$ -- cannot schedule the first half in the pre-lunch of last periods.  
$x_{i,j+1} \geq x_{i,j} \; \forall i \in S, j:Db_j = 1$ -- students must be enrolled in both parts of double. 


$ \sum_{s \in R} r_{j,s,t} = c_{j,t} $ -- if a course is offered, it is given exactly one room
    
$\sum_{j \in C} r_{j,s,t} \leq 1 $ for $s \in R$ and $t \in T$ -- Each room can have at most one class in it at a time (we should make the gym two different rooms).

TODO: Formulate room constraint for departnemnts, i.e., at most 3 science courses per period.

** Objective: **

$ \text{max }\sum_{i \in S} \sum_{j \in C} x_{i,j} P_{i,j} $ -- Assuming preferences take a higher value if they are a student's preferred choice, this will give a higher weight to higher assignments (at this point, I am leaving out the seniority multiplier).

In [39]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd

In [40]:
# read in data
prefs = pd.read_csv("Resources/FlatChoicesBinary.csv")
courses = pd.read_csv("Resources/FlatCourseSize.csv")
#prox = pd.read_csv("Resources/Proximity.csv")
prox = pd.read_csv("Resources/Proximity_1_Diag.csv")

# clean it up
prefs.rename(columns={"Unnamed: 0": "Student"}, inplace=True)
courses.rename(columns={"0":"Class"}, inplace=True)
courses.drop("Unnamed: 0", axis=1, inplace=True)

In [41]:
# Extract sets
S = prefs["Student"].tolist() # list of all students (once we get ID make dictionary)

Cd = {} # Course dictionary
for i in courses.index:
    Cd[i] = courses["Class"].iloc[i]
C = range(len(Cd))
    
T = [1,2,3,4,7,8] # Periods

## STILL NEED INSTRUCTORS

In [42]:
# Extract Preferences
P = prefs.drop("Student", axis=1).as_matrix()

In [43]:
# Double periods
Db = courses["Double"].fillna(0).astype(int)

In [44]:
# Proximity Matrix
D = prox.drop("0", axis=1).as_matrix()

In [45]:
# Course Sizes (min and max)
MIN = courses["Min"]
MAX = courses["Max"]

# To check feasibility:
MIN = [0]*len(C)
MAX = [100]*len(C)

In [46]:
# Need instructor data

In [70]:
# Setup model
m = Model("PhaseTwo")

In [71]:
# Trackers--to verify what SCIP says
num_vars = 0
num_cons = 0

In [72]:
# Add Student Variables (X)
X = {}
for i in S:
    for j in range(len(C)):
        name = "Student " + str(i) + " in course " + str(j)
        X[i,j] = m.addVar(name, vtype='B')
        num_vars += 1

In [73]:
# Add Course Variable
Course = {} # Variable dictionary
for j in range(len(C)):
    for t in T:
        name = "Course " + str(j) + " in period " + str(t)
        Course[j,t] = m.addVar(name, vtype='B')
        num_vars += 1

In [74]:
# Add Student assignment constraint (must have two classes)
## WE CAN ELIMINATE THIS, IF WE FORCE A STUDENT IN ONE COURSE PER PERIOD
# for i in S:
#         m.addCons(quicksum(X[i,j] for j in C) == 6) # one per period
#         num_cons += 1

In [75]:
# Students only given courses they preferenced
for i in S:
    for j in C:
        #m.addCons((1 - X[i,j]) + P[i][j] >= 1)
        m.addCons(X[i,j] <= P[i][j])

In [76]:
# Create the u variable
U = {}
for i in S:
    for j in range(len(C)):
        for t in T:
            name = "min " + str(i) + ", " + str(j) + ", " + str(t)
            U[i,j,t] = m.addVar(name, vtype='B')
            num_vars += 1

In [77]:
# "AND" Constraint--no more than one course per period for a student
for i in S:
    for j in C:
        m.addCons(X[i,j] == quicksum(U[i,j,t] for t in T))
        num_cons += 1
        for t in T:
            m.addCons(Course[j,t] >= U[i,j,t])
            num_cons += 1

In [78]:
# Add capacity and minimum constraint
for j in range(len(C)):
    #m.addCons(quicksum(X[i,j] for i in S) <= MAX[j])
    m.addCons(quicksum(X[i,j] for i in S) <= 100)
    #m.addCons(quicksum(X[i,j] for i in S) >= 0)
    num_cons += 2

# WE NEED PROXIMITY FIX

In [79]:
## REQUIRES TEACHER DATA
# Teacher Constraint
# for k in I:
#     for t in T:
#         m.addCons(quicksum(Course[j,t]*Ta[k][j] for j in C) == 1)
#         num_cons += 1

In [80]:
# Course Taught only once Constraint
for j in range(len(C)):
    m.addCons(quicksum(Course[j,t] for t in T) == 1)
    num_cons += 1

In [81]:
# Double Period--Consecutive Constraint
for j in range(len(C))[:-1]:
    for t in T[:-1]: # need the :-1 to ensure don't go over bounds below
        if t != 4 and t != 8:
            m.addCons(2 - Db[j] - Course[j,t] + Course[j+1, t+1] >= 1)

In [82]:
# Double period--consecutive constraints
for j in range(len(C)):
    if Db[j] == 1: # if double period
        for t in T:
            if t != 4 and t != 8:
                m.addCons(Course[j,t] <= Course[j+1, t+1])
                num_cons += 1

In [83]:
# Double Period--not 4th or 8th
for j in range(len(C)):
    if Db[j] == 1:
        m.addCons(Course[j,4] == 0)
        m.addCons(Course[j,8] == 0)
        num_cons += 2

In [84]:
# Double Period--Student in both
for i in S:
    for j in range(len(C)):
        if Db[j] == 1:
            m.addCons(X[i,j+1] >= X[i,j])
            num_cons += 1

In [85]:
## NEED ROOM DATA
# If course taught, gets one room
# for j in range(len(C)):
#     for t in T:
#         m.addCons(quicksum(r[j,s,t] for s in R) == Course[j,t])

In [86]:
## NEED ROOM DATA
# Room Constraint--Each room gets at most one course
# for s in R:
#     for t in T:
#         m.addCons(quicksum(r[j,s,t] for j in C) <= 1)

In [87]:
## TEMPORARY CONSTAINT
## NO MORE THAN 8 CLASSES PER PERIOD
for t in T:
    m.addCons(quicksum(Course[j,t] for j in C) <= 20)

In [88]:
# Set objective
#m.setObjective(quicksum(X[i,j]*P[i][j] for i in S for j in C), "maximize")
m.setObjective(X[1,1]*0, "maximize") # just find a feasible solution

In [89]:
print(str(num_vars), "Variables")
print(str(num_cons), "Constraints")

189430 Variables
190885 Constraints


In [90]:
# Solve model
m.optimize() # NOTE: solver info printed to terminal

In [68]:
# Print Information on Solve
m.printStatistics() # NOTE: this will only print to terminal (note notebook)

In [91]:
if m.getStatus() == "optimal":
    print("We found an optimal solution!")
else:
    print("The problem is", m.getStatus())

# determine which courses are offered in which period
offered = {}
for t in T:
    class_list = []
    for j in range(len(C)):
        if m.getVal(Course[j,t]) == 1:
            class_list.append(Cd[j])
    offered[t] = class_list

# How many courses per period is each student assigned
for t in T:
    for i in S:
        num_courses = 0
        for j in C:
            if m.getVal(X[i,j]) == 1 and m.getVal(Course[j,t]) ==1:
                num_courses += 1
        print("Student", i, "in", num_courses, "classes in period", t)

We found an optimal solution!
Student 0 in 7 classes in period 1
Student 1 in 8 classes in period 1
Student 2 in 3 classes in period 1
Student 3 in 2 classes in period 1
Student 4 in 6 classes in period 1
Student 5 in 7 classes in period 1
Student 6 in 4 classes in period 1
Student 7 in 2 classes in period 1
Student 8 in 1 classes in period 1
Student 9 in 7 classes in period 1
Student 10 in 9 classes in period 1
Student 11 in 3 classes in period 1
Student 12 in 4 classes in period 1
Student 13 in 1 classes in period 1
Student 14 in 1 classes in period 1
Student 15 in 4 classes in period 1
Student 16 in 4 classes in period 1
Student 17 in 10 classes in period 1
Student 18 in 7 classes in period 1
Student 19 in 6 classes in period 1
Student 20 in 8 classes in period 1
Student 21 in 1 classes in period 1
Student 22 in 9 classes in period 1
Student 23 in 9 classes in period 1
Student 24 in 3 classes in period 1
Student 25 in 7 classes in period 1
Student 26 in 8 classes in period 1
Student

Student 94 in 5 classes in period 2
Student 95 in 6 classes in period 2
Student 96 in 2 classes in period 2
Student 97 in 5 classes in period 2
Student 98 in 4 classes in period 2
Student 99 in 2 classes in period 2
Student 100 in 4 classes in period 2
Student 101 in 2 classes in period 2
Student 102 in 3 classes in period 2
Student 103 in 5 classes in period 2
Student 104 in 8 classes in period 2
Student 105 in 4 classes in period 2
Student 106 in 5 classes in period 2
Student 107 in 4 classes in period 2
Student 108 in 5 classes in period 2
Student 109 in 5 classes in period 2
Student 110 in 4 classes in period 2
Student 111 in 0 classes in period 2
Student 112 in 0 classes in period 2
Student 113 in 4 classes in period 2
Student 114 in 5 classes in period 2
Student 115 in 4 classes in period 2
Student 116 in 3 classes in period 2
Student 117 in 3 classes in period 2
Student 118 in 6 classes in period 2
Student 119 in 3 classes in period 2
Student 120 in 2 classes in period 2
Student

Student 276 in 1 classes in period 3
Student 277 in 4 classes in period 3
Student 278 in 5 classes in period 3
Student 279 in 4 classes in period 3
Student 280 in 3 classes in period 3
Student 281 in 2 classes in period 3
Student 282 in 4 classes in period 3
Student 283 in 1 classes in period 3
Student 0 in 1 classes in period 4
Student 1 in 3 classes in period 4
Student 2 in 2 classes in period 4
Student 3 in 1 classes in period 4
Student 4 in 1 classes in period 4
Student 5 in 1 classes in period 4
Student 6 in 1 classes in period 4
Student 7 in 2 classes in period 4
Student 8 in 3 classes in period 4
Student 9 in 1 classes in period 4
Student 10 in 2 classes in period 4
Student 11 in 3 classes in period 4
Student 12 in 3 classes in period 4
Student 13 in 2 classes in period 4
Student 14 in 2 classes in period 4
Student 15 in 5 classes in period 4
Student 16 in 4 classes in period 4
Student 17 in 0 classes in period 4
Student 18 in 3 classes in period 4
Student 19 in 1 classes in per

Student 237 in 2 classes in period 4
Student 238 in 0 classes in period 4
Student 239 in 1 classes in period 4
Student 240 in 0 classes in period 4
Student 241 in 4 classes in period 4
Student 242 in 0 classes in period 4
Student 243 in 2 classes in period 4
Student 244 in 5 classes in period 4
Student 245 in 1 classes in period 4
Student 246 in 1 classes in period 4
Student 247 in 1 classes in period 4
Student 248 in 1 classes in period 4
Student 249 in 5 classes in period 4
Student 250 in 4 classes in period 4
Student 251 in 1 classes in period 4
Student 252 in 3 classes in period 4
Student 253 in 1 classes in period 4
Student 254 in 1 classes in period 4
Student 255 in 1 classes in period 4
Student 256 in 2 classes in period 4
Student 257 in 1 classes in period 4
Student 258 in 2 classes in period 4
Student 259 in 3 classes in period 4
Student 260 in 0 classes in period 4
Student 261 in 1 classes in period 4
Student 262 in 1 classes in period 4
Student 263 in 1 classes in period 4
S

Student 249 in 0 classes in period 7
Student 250 in 2 classes in period 7
Student 251 in 1 classes in period 7
Student 252 in 2 classes in period 7
Student 253 in 0 classes in period 7
Student 254 in 0 classes in period 7
Student 255 in 1 classes in period 7
Student 256 in 2 classes in period 7
Student 257 in 0 classes in period 7
Student 258 in 0 classes in period 7
Student 259 in 2 classes in period 7
Student 260 in 0 classes in period 7
Student 261 in 0 classes in period 7
Student 262 in 0 classes in period 7
Student 263 in 0 classes in period 7
Student 264 in 2 classes in period 7
Student 265 in 0 classes in period 7
Student 266 in 1 classes in period 7
Student 267 in 0 classes in period 7
Student 268 in 0 classes in period 7
Student 269 in 1 classes in period 7
Student 270 in 0 classes in period 7
Student 271 in 0 classes in period 7
Student 272 in 0 classes in period 7
Student 273 in 1 classes in period 7
Student 274 in 1 classes in period 7
Student 275 in 0 classes in period 7
S

In [92]:
offered

{1: ['Anatomy and Physiology',
  'Asia Studies',
  'English Seminar',
  'Evolutionary Biology',
  'Geology',
  'HS PE',
  'HS PE 2',
  'MS PE',
  'MS/HS PE',
  'MS/HS PE 2',
  'MS/HS PE 3',
  'Other',
  'Other 2',
  'Other 3',
  'Other 4',
  'Other 5',
  'Other 6',
  'Probability and Statistics',
  'Researchers (8th Grade SS)',
  'Spanish A'],
 2: ['Advanced Algebra and Trigonometry 2',
  'Algebra A',
  'Banned Books',
  'Beginning Algebra and Trigonometry',
  'Celtic Band',
  'Choir',
  'Dark Fiction',
  'Empty',
  'HS English',
  'HS English 2',
  'HS Health',
  'HS PE 3',
  'I Know Some Things',
  'MS Social Studies',
  'MS Social Studies 2',
  'Music, Myth, and Ritual',
  "Poet's Society",
  'Spanish A 2',
  'Spanish B',
  'Spanish B 2'],
 3: ['6th Grade Art',
  '6thGradeOther',
  'Advanced Algebra and Trigonometry',
  'Algebra A 2',
  'Algebra B',
  'Algebra B 2',
  'Community Service Class',
  'Computer Literacy',
  'Drawing and Painting',
  'Economics',
  'French A',
  'Intermed

In [88]:
for i in S:
    s = 0
    for j in C:
        a = m.getVal(U[i,j,1])
        s += a
    print("Studnet", i, "sum is", s)

Studnet 0 sum is 1.0
Studnet 1 sum is 1.0
Studnet 2 sum is 1.0
Studnet 3 sum is 1.0
Studnet 4 sum is 1.0
Studnet 5 sum is 1.0
Studnet 6 sum is 1.0
Studnet 7 sum is 1.0
Studnet 8 sum is 1.0
Studnet 9 sum is 1.0
Studnet 10 sum is 1.0
Studnet 11 sum is 1.0
Studnet 12 sum is 1.0
Studnet 13 sum is 1.0
Studnet 14 sum is 1.0
Studnet 15 sum is 1.0
Studnet 16 sum is 1.0
Studnet 17 sum is 1.0
Studnet 18 sum is 1.0
Studnet 19 sum is 1.0
Studnet 20 sum is 1.0
Studnet 21 sum is 1.0
Studnet 22 sum is 1.0
Studnet 23 sum is 1.0
Studnet 24 sum is 1.0
Studnet 25 sum is 1.0
Studnet 26 sum is 1.0
Studnet 27 sum is 1.0
Studnet 28 sum is 1.0
Studnet 29 sum is 1.0
Studnet 30 sum is 1.0
Studnet 31 sum is 1.0
Studnet 32 sum is 1.0
Studnet 33 sum is 1.0
Studnet 34 sum is 1.0
Studnet 35 sum is 1.0
Studnet 36 sum is 1.0
Studnet 37 sum is 1.0
Studnet 38 sum is 1.0
Studnet 39 sum is 1.0
Studnet 40 sum is 1.0
Studnet 41 sum is 1.0
Studnet 42 sum is 1.0
Studnet 43 sum is 1.0
Studnet 44 sum is 1.0
Studnet 45 sum is 1.

## Current issues:
- Missing data on:
    - Teachers
    - Course Proximity (departments)
    - Updates to accomodate:
        - Double period courses
        - Multiple instances of same course
        - "Other" in every peoriod
- Proximity constraint may be over complicated (the double sum involving $D$)
    

# TODO
- Room constraints for departments, i.e., no more than 3 science courses per period. 
- Code all Room and Double period constraints
- Enhance output of model to handle large-scale data

The following block of code is meant to help better understand SCIP. It instantiates a Model instance, then parses through each of its public methods and fields, looking for, and printing, their docstrings.

In [8]:
# Figure out wtf SCIP is doing

# initialize a Model instance
mod = Model("what?")

# get methods for the model
methods = dir(mod)
i = 0
for x in methods:
    if x[0]=="_":
        i = methods.index(x)
methods = methods[i+1:] # only want the public methods

# print out each method and its info
for m in methods:
    print(m + ":")
    print(getattr(mod, m).__doc__)
    print("\n")

addCons:
Add a linear or quadratic constraint.

        Keyword arguments:
        cons -- list of coefficients
        name -- the name of the constraint (use generic name if empty)
        initial -- should the LP relaxation of constraint be in the initial LP? (default True)
        separate -- should the constraint be separated during LP processing? (default True)
        enforce -- should the constraint be enforced during node processing? (default True)
        check -- should the constraint be checked for feasibility? (default True)
        propagate -- should the constraint be propagated during node processing? (default True)
        local -- is the constraint only valid locally? (default False)
        modifiable -- is the constraint modifiable (subject to column generation)? (default False)
        dynamic -- is the constraint subject to aging? (default False)
        removable -- hould the relaxation be removed from the LP due to aging or cleanup? (default False)
        stick

In [6]:
import pyscipopt as scip

In [8]:
dir(scip)

['Branchrule',
 'Conshdlr',
 'Heur',
 'LP',
 'Model',
 'Multidict',
 'Presol',
 'Pricer',
 'Prop',
 'SCIP_HEURTIMING',
 'SCIP_PARAMEMPHASIS',
 'SCIP_PARAMSETTING',
 'SCIP_PRESOLTIMING',
 'SCIP_PROPTIMING',
 'SCIP_RESULT',
 'SCIP_STAGE',
 'SCIP_STATUS',
 'Sepa',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'multidict',
 'quicksum',
 'scip']